In [1]:
import pandas as pd
df_train = pd.read_csv('df_jan_train.csv', engine='python')

In [2]:
sat_id = 123
cols = ['epoch','x','y','z']
df = df_train[df_train.sat_id==sat_id][cols]
df['epoch'] = pd.to_datetime(df['epoch'], infer_datetime_format=True)
df.reset_index(drop=True, inplace=True)

In [3]:
df.head()

epoch             x             y            z
0 2014-01-01 00:00:00.000  14511.024670  13023.085484 -3929.568361
1 2014-01-01 00:29:40.162  19297.897633   6014.609551 -1580.558834
2 2014-01-01 00:59:20.323  21225.256604  -1908.309300  1010.219760
3 2014-01-01 01:29:00.485  20427.312944  -9617.121972  3481.221462
4 2014-01-01 01:58:40.647  17470.559731 -16340.560769  5594.213173

In [4]:
import statistics

def get_values(i, df, col):
    v1 = df.loc[i-1,col]
    v2 = df.loc[i,col]
    return statistics.median([v1,v2])

def get_complete_df(df):
    
    dfs = []
    for i in range(1,len(df)):
        
        t1 = df.loc[i-1,'epoch']
        t2 = df.loc[i,'epoch']
        tm = pd.Series([t1,t2]).median()
        
        ok = len(set([t1.minute,t2.minute,tm.minute]))==3
        
        if ok:
            xm = get_values(i, df, 'x')
            ym = get_values(i, df, 'y')
            zm = get_values(i, df, 'z')

            dfs.append(pd.DataFrame([[tm,xm,ym,zm]], columns=cols))

    return ok, pd.concat([df, pd.concat(dfs)]).sort_values(by='epoch').reset_index(drop=True)

ok = True
while ok:
    ok, df = get_complete_df(df)

df.head()

epoch            x             y            z
0 2014-01-01 00:00:00.000000000  14511.02467  13023.085484 -3929.568361
1 2014-01-01 00:01:51.260124928  14810.20423  12585.055738 -3782.755265
2 2014-01-01 00:02:46.890187264  14959.79401  12366.040866 -3709.348718
3 2014-01-01 00:03:42.520249856  15109.38379  12147.025993 -3635.942170
4 2014-01-01 00:04:38.150312448  15258.97357  11928.011120 -3562.535622

In [5]:
# extraigo el día y la hora de la columna epoch
df = df.assign(**{t:getattr(df.epoch.dt,t) for t in ['day','hour']})

# calculo los minutos transcurridos de cada horario con respecto al comienzo del día
df['min_aux'] = df.epoch.dt.strftime('%H:%M').str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

# acumulo los minutos transcurridos desde el las 00 del primer día de medición (1-ene)
df['minute'] = df.apply(lambda x: x.min_aux + 60 * 24 * (x.day - 1), axis=1)

# selecciono las columnas
df = df[['minute','x','y','z']]

In [6]:
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks

minute = df['minute']
x = df['x']
y = df['y']
z = df['z']
print(len(minute),len(x),len(y),len(z))
peaks, _ = find_peaks(x, height=0)

rounds = pd.Series([len(peaks)]*len(minute))
for p, peak in reversed(list(enumerate(peaks))):
    print(p, peak)
    rounds[:peak] = p

min_p = min(peaks)
max_p = max(peaks)
print(min_p,max_p)
x = x[min_p:max_p]
y = y[min_p:max_p]
z = z[min_p:max_p]
cicles = rounds[min_p:max_p]
minute = minute[min_p:max_p]
print(len(minute),len(x),len(y),len(z),len(cicles))

filename = f'ml/sat_id_{sat_id}_by_cicles_median.csv'
columns = ['minute', 'cicle', 'x', 'y', 'z']
pd.DataFrame(list(zip(minute, cicles, x, y, z)), columns=columns).sort_values(by='minute').to_csv(filename, index=False)

34449 34449 34449 34449
48 34181
47 33469
46 32757
45 32045
44 31333
43 30621
42 29909
41 29196
40 28484
39 27772
38 27060
37 26348
36 25636
35 24923
34 24211
33 23499
32 22787
31 22075
30 21363
29 20651
28 19939
27 19227
26 18515
25 17802
24 17090
23 16378
22 15666
21 14954
20 14242
19 13530
18 12818
17 12106
16 11394
15 10682
14 10028
13 9316
12 8604
11 7892
10 7179
9 6467
8 5755
7 5043
6 4331
5 3619
4 2907
3 2195
2 1483
1 771
0 59
59 34181
34122 34122 34122 34122 34122


In [7]:
pd.read_csv(f'ml/sat_id_{sat_id}_by_cicles_median.csv', engine='python').groupby('cicle').agg({'minute':'count'}).reset_index()

cicle  minute
0       1     712
1       2     712
2       3     712
3       4     712
4       5     712
5       6     712
6       7     712
7       8     712
8       9     712
9      10     712
10     11     713
11     12     712
12     13     712
13     14     712
14     15     654
15     16     712
16     17     712
17     18     712
18     19     712
19     20     712
20     21     712
21     22     712
22     23     712
23     24     712
24     25     712
25     26     713
26     27     712
27     28     712
28     29     712
29     30     712
30     31     712
31     32     712
32     33     712
33     34     712
34     35     712
35     36     713
36     37     712
37     38     712
38     39     712
39     40     712
40     41     712
41     42     713
42     43     712
43     44     712
44     45     712
45     46     712
46     47     712
47     48     712

In [8]:
pd.read_csv(f'by_cicle/sat_id_{sat_id}_by_cicles.csv', engine='python').groupby('cicle').agg({'minute':'count'}).reset_index()

cicle  minute
0       1      24
1       2      24
2       3      24
3       4      24
4       5      24
5       6      24
6       7      24
7       8      24
8       9      24
9      10      24
10     11      24
11     12      24
12     13      24
13     14      24
14     15      24
15     16      24
16     17      24
17     18      24
18     19      24
19     20      24
20     21      24
21     22      24
22     23      24
23     24      24
24     25      24
25     26      24
26     27      24
27     28      24
28     29      24
29     30      24
30     31      24
31     32      24
32     33      24
33     34      24
34     35      24
35     36      24
36     37      24
37     38      24
38     39      24
39     40      24
40     41      24
41     42      24
42     43      24
43     44      24
44     45      24
45     46      24
46     47      24
47     48      24

In [ ]:
df['rank_t'] = range(0, len(df), 1)
df['rank_f'] = (df['rank_t'] + 1).astype(int)

df = pd.merge(df[df.columns.difference(['rank_t'])], df[['rank_t','x','y','z']], left_on=['rank_f'], right_on=['rank_t'], how='inner')
df.rename(columns={'x_y':'next_x','y_y':'next_y','z_y':'next_z','x_x':'x','y_x':'y','z_x':'z'}, inplace=True)
df = df[['epoch','x','y','z','next_x','next_y','next_z']]
df.head()

In [ ]:
import matplotlib.pylab as plt

from pyFTS.partitioners import Grid, Util as pUtil
from pyFTS.common import Transformations
from pyFTS.models.multivariate import common, variable, mvfts
from pyFTS.models.seasonal import partitioner as seasonal
from pyFTS.models.seasonal.common import DateTime

bc = Transformations.BoxCox(0)
diff = Transformations.Differential(1)

np = 10
target_col = 'next_x'

model = mvfts.MVFTS()

fig, axes = plt.subplots(nrows=5, ncols=1,figsize=[15,10])

sp = {'seasonality': DateTime.minute_of_day}

vhour = variable.Variable("Hour", data_label="epoch", partitioner=seasonal.TimeGridPartitioner, npart=24, 
                          data=df, partitioner_specific=sp)
model.append_variable(vhour)
vhour.partitioner.plot(axes[0])

v_x = variable.Variable("X", data_label="x", partitioner=Grid.GridPartitioner, npart=np, data=df)
model.append_variable(v_x)
v_x.partitioner.plot(axes[1])

v_y = variable.Variable("Y", data_label="y", partitioner=Grid.GridPartitioner, npart=np, data=df)
model.append_variable(v_y)
v_y.partitioner.plot(axes[2])

v_z = variable.Variable("Z", data_label="z", partitioner=Grid.GridPartitioner, npart=np, data=df)
model.append_variable(v_z)
v_z.partitioner.plot(axes[3])

v_target = variable.Variable("Target", data_label=target_col, partitioner=Grid.GridPartitioner, npart=20, data=df)
#model.append_variable(v_target)
v_target.partitioner.plot(axes[4])

model.target_variable = v_target

model.fit(df)

plt.tight_layout()

In [ ]:
forecasts = model.predict(df.iloc[1000:10000,:])
target = df['next_x'].iloc[1000:10000].to_list()

In [ ]:
ig = plt.figure(figsize=(30,30))
plt.plot(forecasts,"r",marker="o")
plt.plot(target,"b",marker="o")
plt.grid()
plt.yticks(range(-31000, 22000, 1000))
plt.show()

In [ ]:
from tabulate import tabulate
data_by_cicle = pd.read_csv(filename, engine='python').head()    
print('SAT_ID', sat_id, len(data_by_cicle), data_by_cicle.minute.diff().mean(), data_by_cicle.minute.diff().std())
print(tabulate(data_by_cicle.head(16), headers=columns, showindex=False, tablefmt="psql"))

In [ ]:
data_by_cicle.cicle.value_counts()

In [ ]:
def get_xy(df, target_col):
    target_cols = ['next_x','next_y','next_z']
    X = df.drop(target_cols, axis=1).values
    y = df[target_col].values
    feature_names = df.drop(target_cols, axis=1).columns.tolist()
    return X, y, feature_names

X_next_x, y_next_x, feature_names = get_xy(df_1, 'next_x')
X_next_y, y_next_y, feature_names = get_xy(df_1, 'next_y')
X_next_z, y_next_z, feature_names = get_xy(df_1, 'next_z')

In [ ]:
## call model
from sklearn import linear_model
model = linear_model.LinearRegression()

## train
model.fit(X_next_x, y_next_x)

## test
predicted = model.predict(X_next_x)

In [ ]:
y_next_x

In [ ]:
predicted

In [ ]:
5.23/2

In [ ]:
model.predict([[5.23/2,10390.313089,-2796.458271,3179.562085]])

In [ ]:
10591.636525	-827.680211	2174.902416